## 7. Compute-EW-for-CIV.ipynb

This notebook computes the EW for the CIV doublet using
create_gaussian_fitter() and calculate_ew_from_gaussian() 
fitter is based on astropy models.Const1D() and models.Gaussian1D()
    
This code just loops through the pickle files for each sightline, observation, redshift


Flow
* using list of the pickle files, sort them and load them
* get list of sightlines
* get list of ions to analyze
* git list of redshifts in pickle files
* git sightlines, redshifts and ios to analyze
* following the flow in 5. Plot-H-CIV.ipynb
* ....


In [6]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import pickle

# need some of my own functions
import os, sys
base = '/Users/robertseaton/School/github_repos/CGM-learning/code'
if base not in sys.path:
    sys.path.insert(0, os.path.abspath(base))
from bobutils import data_analysis as da

import notebook_utils as nbu # in the current directory


In [45]:
def plot_single_ion_panel(panel_ax, ion_data, fit_data, use_vel,
                          label="SL:X", fontsize=12, is_top=True, is_bottom=True):
    
    #---------------Define variables for readability--------------#
    vel = ion_data['vel']
    wave = ion_data['wave']
    error = ion_data['error']
    flux = ion_data['flux']
    weight = ion_data['weight']
    name = ion_data['name']
    #L = ion_data.L
    wc =np.array(ion_data['wc']&(error != 0)) #error != 0 is a bad pixel mask 
    cont = ion_data['cont']
    window_lim = ion_data['window_lim']
    order = ion_data['order']
    EWlims = ion_data['EWlims']
    EW_text = ion_data['text']
    
    lam_0=ion_data['lam_0']
    fvals=ion_data['f']
    z_str= ion_data['z']
    zabs = float(z_str)

    # -- from fit_data
    wlims = fit_data.wlims
    ewlims_rest = fit_data.ewlims_rest
    wingspan = fit_data.wingspan
    xdata = fit_data.xdata
    nflux = fit_data.nflux
    g = fit_data.g
    fit_g = fit_data.fit_g
    ew = fit_data.ew
    ew_sig = fit_data.ew_sig
    good_fit = fit_data.good_fit
    # more data
    # g, ew, ew_sig, good_fit,                              # fit_and_compute_ew:                   g, fit_g, ew, ew_sig, good_fit
    # xdata, nflux,  wlims, ewlims_rest, wingspan, nerror,  # create_metadata_for_gaussian_fitting: xdata, nflux, nerror, gxdata_rest, gnflux, gnerror, ewlims_rest, wlims, wingspan
    # print(f"{label}: {zabs=}  :  {good_fit=}")
    
    wlims_rest = [wlims[0]/(1. + zabs),wlims[1]/(1. + zabs)]
    fit_xdata = np.linspace(wlims_rest[0], wlims_rest[1], 1000) # just gives the fit a smooth look

    panel_ax.clear()
    
    #-------------------------- begin drawing -------------------------#
    if is_top: panel_ax.set_title('Normalized Spectra')
    if is_bottom: panel_ax.set_xlabel('Velocity (km/s)')
    
    panel_ax.step(vel/(1. + zabs),flux/cont, color='k',where='mid')
    panel_ax.step(vel/(1. + zabs),error/cont, color='r',where='mid')
    
    if wc[0] == True:
        gray_idx = np.where(np.diff(wc,prepend=np.nan))[0][1:]
    else:
        gray_idx = np.where(np.diff(wc,prepend=np.nan))[0]    
    # print(f"{gray_idx=}")
    for zz in range(int(len(gray_idx)/2)):
        #gray_idx = gray_idx-1
        panel_ax.step(vel[gray_idx[zz*2]:gray_idx[2*zz+1]]/(1. + zabs),
                      flux[gray_idx[2*zz]:gray_idx[2*zz+1]],
                      vel[gray_idx[2*zz]:gray_idx[2*zz+1]]/(1. + zabs),
                      error[gray_idx[2*zz]:gray_idx[2*zz+1]],
                      where='mid',color='lightgray',linewidth=1.3,alpha=1)
    
    panel_ax.plot(xdata/(1. + zabs), nflux, 'o', color="tab:red", label="flux", linewidth=1.0, markersize=2) # data points
    panel_ax.plot(fit_xdata, g(fit_xdata), 
            '-', 
            color="tab:green", 
            linewidth=3.0, alpha=0.5,
            markersize=3)
    continuum = g.parameters[0]

    panel_ax.fill_between(fit_xdata, continuum, g(fit_xdata), alpha=0.7)

    panel_ax.axhline(y=1,xmin=window_lim[0],xmax=window_lim[1],ls='--',c='b')
    
    panel_ax.set_ylim([0,2.2])
    
    # panel_ax.axvspan(ewlims_rest[0]-wingspan, ewlims_rest[1]+wingspan, alpha=0.2, color='tab:gray')

    flux_max = np.max(np.abs(nflux))
    flux_min = np.min(np.abs(nflux))
    panel_ax.vlines(ewlims_rest, flux_min, flux_max, color="tab:red") 

    panel_ax.set_xlim(wlims_rest)
    
    panel_ax.axvline(0,ymin=0,ymax=panel_ax.get_ylim()[1],ls='--',c='k')

    #Plot EW velocity limit
    if EWlims[0] is not None: panel_ax.axvline(EWlims[0]/(1. + zabs),ymin=0,ymax=2.5,ls='--',c='b')
    if EWlims[1] is not None: panel_ax.axvline(EWlims[1]/(1. + zabs),ymin=0,ymax=2.5,ls='--',c='r')

    pm = r"$\pm$"
    mA = r"$m\AA$"
    good_fit_str = "Yes" if good_fit else "No"
    left_side_info = f"""EW (init): {EW_text}
EW (gauss): {fit_data.ew:0.3g} {pm} {fit_data.ew_sig:0.3g} {mA}
Good fit: {good_fit_str}
"""
    panel_ax.text(0.05,0.95,left_side_info,
              horizontalalignment='left',
              verticalalignment='top',                  
              transform=panel_ax.transAxes,color='black')       
    
    panel_ax.text(0.95,0.95,label,
              horizontalalignment='right',
              verticalalignment='top',
              transform=panel_ax.transAxes,color='black')
    
    plt.subplots_adjust(wspace=0.2)
    plt.draw()

In [46]:
def init_figure(panel_title, panel_count, figsize=(8, 2), dpi=100):
    """ create the matplotlib axes we'll draw our panels in """

    sns.set_theme()
    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.suptitle(panel_title, size=22.0, weight=0.5)

    fig.set_figheight(8)
    fig.set_figwidth(20)    
    cols = 2
    rows = int((panel_count + 1) / cols)
    spec = gridspec.GridSpec(ncols=cols, nrows=rows, width_ratios=[2, 2], wspace=0.2, hspace=0.2)
    ax = []
    for i in range(panel_count):
        ax.append(fig.add_subplot(spec[i]))
    
    return fig, ax


In [47]:
def display_selected_ion_panel(selected_ion=None, ion_data=None):
    if ion_data is None or selected_ion is None:
        print("Must provide selected_ion and ion_data")
        return
    
    use_vel = True
    
    for zabs_string in list(ion_data.keys()):
        zabs = float(zabs_string)

        print(f"""
------------------------------------------------------------
zabs: {zabs}""")
        sl_cnt = nbu.count_sightlines(zabs, ion_data)
        fig, ax_list = init_figure(f"zabs: {zabs}", sl_cnt)
    
        for ii, sl in enumerate(ion_data[zabs]):
            ions = ion_data[zabs][sl]
        
            for ion in ions:
                # if ion is not None and (ion == 'CIV 1548' or ion == 'CIV 1550' or ion == 'HI 1215'):
                if ion is not None and (ion == selected_ion):
                    fig.suptitle(f"{ion}    zabs: {zabs}", size=22.0, weight=0.5)

                    if ii < 2: is_top=True
                    else: is_top = False
                    if ii >= 4: is_bottom = True # need a more data-driven way to determine this
                    else: is_bottom = False

                    fit_data = nbu.fit_ion(ions[ion], use_vel)
                    plot_single_ion_panel(ax_list[ii], ions[ion], 
                                          fit_data, use_vel,
                                          label=f"Sightline: {sl}", is_top=is_top, is_bottom=is_bottom)

In [48]:
def display_CIV_1548_1550(ion_data):
    display_selected_ion_panel(selected_ion='CIV 1548', ion_data=ion_data)
    display_selected_ion_panel(selected_ion='CIV 1550', ion_data=ion_data)

In [49]:
def display_select_CIV_sl_zabs(ion_data, sl, zabs):
    """ display CIV 1548 and CIV 1550 at a specified redshift and sightline """
    for zabs_string in list(ion_data.keys()):
        zabs = float(zabs_string)
        if zabs == zabs:
            print(f"""
------------------------------------------------------------
zabs: {zabs}""")

In [81]:
def display_selected_ion_zabs_sl(ion_data=None, selected_ion=None, selected_zabs=None, selected_sl=None):
    if ion_data is None or selected_ion is None or selected_zabs is None or selected_sl is None:
        print("Must provide ion_data, selected_ion, selected_zabs, and selected_sl")
        return
    
    use_vel = True
    
    zabs = float(selected_zabs)
    fig, ax_list = init_figure(f"zabs: {zabs}", 2)
    
    sl = int(selected_sl)
    ions = ion_data[zabs][selected_sl]
    ion = ions[selected_ion]
#     print(f"""{ion=}
# """)

    fig.suptitle(f"{ion}    zabs: {zabs}", size=22.0, weight=0.5)

    fit_data = nbu.fit_ion(ion, use_vel)
    # plot_single_ion_panel(ax_list[0], {ion}, 
    #                         fit_data, use_vel,
    #                         label=f"Sightline: {sl}", is_top=True, is_bottom=True)

In [82]:
def main_old():
    ion_data = nbu.load_pickled_ion_data()
    display_selected_ion_panel(selected_ion="CIV 1548", ion_data=ion_data)  
    display_selected_ion_panel(selected_ion="CIV 1550", ion_data=ion_data)  
    
def show_pickled_ion_data(ion_data):
    for z in ion_data:
        print(f"{z=}")
        sightlines = ion_data[z]
        for sl in sightlines:
            print(f"  {sl=}")
            for ion in sightlines[sl]:
                # print(f"    {ion=}: {sightlines[sl][ion].keys()}")
                if ion is not None and (ion == 'CIV 1548' or ion == 'CIV 1550' or ion == 'HI 1215'):
                    print(f"    {ion=}")
                    
def main():
    ion_data = nbu.load_pickled_ion_data()
    # show_pickled_ion_data(ion_data)
    display_selected_ion_zabs_sl(ion_data=ion_data, 
                                 selected_ion='CIV 1548', 
                                 selected_zabs=2.1813, 
                                 selected_sl='4')


if __name__ == '__main__':
    main()

Inside calculate_ew_from_gaussian  ----- 
g=<Gaussian1D(amplitude=0.60566437, mean=-11.01269487, stddev=0.)>


ValueError: 
\\pm
^
Expected main, found '\'  (at char 0), (line:1, col:1)

<Figure size 2000x800 with 2 Axes>